In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Author: Milan Ondrasovic <milan.ondrasovic@gmail.com>

from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

import cv2 as cv
import numpy as np

import matplotlib.pyplot as plt

from PIL import Image, ImageStat, ImageOps

from sot.bbox import BBox
from sot.utils import Size

plt.style.use('tableau-colorblind10')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 11

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
my_model_file_path = "../model_siamfc.pth"
my_model = torch.load(my_model_file_path, map_location=torch.device('cpu'))

new_model_file_path = "../siamfc_alexnet_e50.pth"
new_model = torch.load(new_model_file_path, map_location=torch.device('cpu'))

len(my_model), len(new_model)

(30, 30)

In [24]:
for my_item, new_item in zip(my_model.items(), new_model.items()):
    print("*" * 80)
    print(f"M: {my_item[0]} {list(my_item[1].shape)}")
    print(f"N: {new_item[0]} {list(new_item[1].shape)}")

********************************************************************************
M: conv1.block.0.weight [96, 3, 11, 11]
N: backbone.conv1.0.weight [96, 3, 11, 11]
********************************************************************************
M: conv1.block.0.bias [96]
N: backbone.conv1.0.bias [96]
********************************************************************************
M: conv1.block.1.weight [96]
N: backbone.conv1.1.weight [96]
********************************************************************************
M: conv1.block.1.bias [96]
N: backbone.conv1.1.bias [96]
********************************************************************************
M: conv1.block.1.running_mean [96]
N: backbone.conv1.1.running_mean [96]
********************************************************************************
M: conv1.block.1.running_var [96]
N: backbone.conv1.1.running_var [96]
********************************************************************************
M: conv1.block.1.num_batches_tr